# NLP - Sentiment Classification


## Overview/Context
- Generate Word Embedding and retrieve outputs of each layer with Keras based on the Classification task.
- Word embedding are a type of word representation that allows words with similar meaning to have a similar representation.
- It is a distributed representation for the text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.
- We will use the IMDb dataset to learn word embedding as we train our dataset.
- This dataset contains 25,000 movie reviews from IMDB, labeled with a sentiment (positive or negative).

## About the Data
- The Dataset of 25,000 movie reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers).
- For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word.
- Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.
- As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.

### Mount Google Drive

In [3]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Setting the current working directory
import os; os.chdir('drive/My Drive/Projects/NLP/Sentiment Classification')

## Import Required Packages

In [5]:
# Import packages
import pandas as pd, numpy as np
import tensorflow as tf
assert tf.__version__ >= '2.0'

from itertools import islice

# Keras
from keras.layers import Dense, Embedding, LSTM, Dropout, MaxPooling1D, Conv1D
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.preprocessing import sequence
from keras.datasets import imdb

from keras.callbacks import ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Suppress warnings
import warnings; warnings.filterwarnings('ignore')

random_state = 40
np.random.seed(random_state)
tf.random.set_seed(random_state)

### Loading the dataset 

In [7]:
from keras.datasets import imdb

vocab_size = 10000 #vocab size

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size) # vocab_size is no.of words to consider from the dataset, ordering based on frequency.

In [ ]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

## Train test split 

In [ ]:
#load dataset as a list of ints
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
#make all sequences of the same length
x_train = pad_sequences(x_train, maxlen = maxlen, padding = 'pre')
x_test =  pad_sequences(x_test, maxlen = maxlen, padding = 'pre')

In [ ]:
X = np.concatenate((x_train, x_test), axis = 0)
y = np.concatenate((y_train, y_test), axis = 0)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_state, shuffle = True)
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state = random_state, shuffle = True)

In [ ]:
print('---'*20, f'\nNumber of rows in training dataset: {x_train.shape[0]}')
print(f'Number of columns in training dataset: {x_train.shape[1]}')
print(f'Number of unique words in training dataset: {len(np.unique(np.hstack(x_train)))}')


print('---'*20, f'\nNumber of rows in validation dataset: {x_valid.shape[0]}')
print(f'Number of columns in validation dataset: {x_valid.shape[1]}')
print(f'Number of unique words in validation dataset: {len(np.unique(np.hstack(x_valid)))}')


print('---'*20, f'\nNumber of rows in test dataset: {x_test.shape[0]}')
print(f'Number of columns in test dataset: {x_test.shape[1]}')
print(f'Number of unique words in test dataset: {len(np.unique(np.hstack(x_test)))}')


print('---'*20, f'\nUnique Categories: {np.unique(y_train), np.unique(y_valid), np.unique(y_test)}')

------------------------------------------------------------ 
Number of rows in training dataset: 32000
Number of columns in training dataset: 300
Number of unique words in training dataset: 9999
------------------------------------------------------------ 
Number of rows in validation dataset: 8000
Number of columns in validation dataset: 300
Number of unique words in validation dataset: 9986
------------------------------------------------------------ 
Number of rows in test dataset: 10000
Number of columns in test dataset: 300
Number of unique words in test dataset: 9986
------------------------------------------------------------ 
Unique Categories: (array([0, 1]), array([0, 1]), array([0, 1]))


##Get word index and create a key-value pair for word and word id


In [ ]:
 def decode_review(x, y):
  w2i = imdb.get_word_index()                                
  w2i = {k:(v + 3) for k, v in w2i.items()}
  w2i['<PAD>'] = 0
  w2i['<START>'] = 1
  w2i['<UNK>'] = 2
  i2w = {i: w for w, i in w2i.items()}

  ws = (' '.join(i2w[i] for i in x))
  print(f'Review: {ws}')
  print(f'Actual Sentiment: {y}')
  return w2i, i2w

w2i, i2w = decode_review(x_train[0], y_train[0])

# get first 50 key, value pairs from id to word dictionary
print('---'*30, '\n', list(islice(i2w.items(), 0, 50)))

Review: <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> i first caught up with jennifer years ago while out of town when it showed up on tv in the middle of the night i fell asleep before it ended but it stuck with me until i had to track it down its appeal is that though there's not a lot to it it weaves an intriguing atmosphere and because <UNK> <UNK> and howard duff real life man and wife at the time display an <UNK> low key chemistry <UNK> plays a woman engaged to house sit a vast california estate whose previous caretaker jennifer up and disappeared shades of jack nicholson in the shining although in this instance it's not <UNK> who goes or went mad duff is the guy in town who manages the <UNK> <UNK> and takes a shine to <UNK> who decide

## Build Keras Embedding Layer Model (30 points)
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 256, input_length = maxlen))
model.add(Dropout(0.25))
model.add(Conv1D(256, 5, padding = 'same', activation = 'relu', strides = 1))
model.add(Conv1D(128, 5, padding = 'same', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size = 2))
model.add(Conv1D(64, 5, padding = 'same', activation = 'relu', strides = 1))
model.add(MaxPooling1D(pool_size = 2))
model.add(LSTM(75))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

# Adding callbacks
es = EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 0)  
mc = ModelCheckpoint('imdb_model.h5', monitor = 'val_loss', mode = 'min', save_best_only = True, verbose = 1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 256)          2560000   
_________________________________________________________________
dropout (Dropout)            (None, 300, 256)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 300, 256)          327936    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 300, 128)          163968    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 150, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 150, 64)           41024     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 75, 64)            0

In [ ]:
# Fit the model
model.fit(x_train, y_train, validation_data = (x_valid, y_valid), epochs = 3, batch_size = 64, verbose = True, callbacks = [es, mc])

# Evaluate the model
scores = model.evaluate(x_test, y_test, batch_size = 64)
print('Test accuracy: %.2f%%' % (scores[1]*100))

Epoch 1/3
500/500 [==============================] - 62s 57ms/step - loss: 0.4776 - accuracy: 0.7214 - val_loss: 0.2563 - val_accuracy: 0.8946

Epoch 00001: val_loss improved from inf to 0.25631, saving model to imdb_model.h5
Epoch 2/3
500/500 [==============================] - 28s 57ms/step - loss: 0.1772 - accuracy: 0.9332 - val_loss: 0.2599 - val_accuracy: 0.8950

Epoch 00002: val_loss did not improve from 0.25631
Epoch 00002: early stopping
157/157 [==============================] - 2s 14ms/step - loss: 0.2443 - accuracy: 0.9026
Test accuracy: 90.26%


In [ ]:
y_pred = model.predict_classes(x_test)
print(f'Classification Report:\n{classification_report(y_pred, y_test)}')

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.88      0.90      5208
           1       0.87      0.93      0.90      4792

    accuracy                           0.90     10000
   macro avg       0.90      0.90      0.90     10000
weighted avg       0.90      0.90      0.90     10000



## Accuracy of the model  & Retrive the output of each layer in keras for a given single test sample from the trained model you built 

In [ ]:
sample_x_test = x_test[np.random.randint(10000)]
for layer in model.layers:

    model_layer = Model(inputs = model.input, outputs = model.get_layer(layer.name).output)
    output = model_layer.predict(sample_x_test.reshape(1,-1))
    print('\n','--'*20, layer.name, 'layer', '--'*20, '\n')
    print(output)


 ---------------------------------------- embedding layer ---------------------------------------- 

[[[-0.00984982  0.02402639  0.00732345 ...  0.01935286  0.0317665
   -0.00632548]
  [-0.00984982  0.02402639  0.00732345 ...  0.01935286  0.0317665
   -0.00632548]
  [-0.00984982  0.02402639  0.00732345 ...  0.01935286  0.0317665
   -0.00632548]
  ...
  [-0.02441969 -0.00658704 -0.05887107 ... -0.01739137  0.04956373
    0.00547364]
  [-0.0294162   0.02436043  0.01181066 ...  0.03686921 -0.04653443
    0.0048245 ]
  [ 0.02370638 -0.01012601  0.11731517 ...  0.09650249 -0.02937531
    0.06121516]]]

 ---------------------------------------- dropout layer ---------------------------------------- 

[[[-0.00984982  0.02402639  0.00732345 ...  0.01935286  0.0317665
   -0.00632548]
  [-0.00984982  0.02402639  0.00732345 ...  0.01935286  0.0317665
   -0.00632548]
  [-0.00984982  0.02402639  0.00732345 ...  0.01935286  0.0317665
   -0.00632548]
  ...
  [-0.02441969 -0.00658704 -0.05887107 ... 

In [ ]:
decode_review(x_test[25], y_test[25])
print(f'Predicted sentiment: {y_pred[25][0]}')

Review: <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <START> this is a movie about the music that is currently being played in <UNK> <UNK> was the center of the two old world <UNK> the <UNK> empire and the <UNK> empire today it is a <UNK> of almost 10 million so it is to no ones surprise that a lot of music is being played in <UNK> with a great variety of voices styles and influences from everywhere on the globe it is turkish music of course and i was fascinated by turkish music ever since i bought my first record long time ago the movie features different singers <UNK> and bands spoken comments from the musicians nicely illustra

#Conclusion
Sentiment classification task on the IMDB dataset, on test dataset,
Accuracy: > 90%

*   F1-score: > 90%
*   Loss of 0.25